<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Useful-Scripts" data-toc-modified-id="Useful-Scripts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Useful Scripts</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Random-Forest-Modelling" data-toc-modified-id="Random-Forest-Modelling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Random Forest Modelling</a></span></li><li><span><a href="#Grid-Search" data-toc-modified-id="Grid-Search-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Grid Search</a></span></li><li><span><a href="#Use-the-best-parameters-from-grid-search" data-toc-modified-id="Use-the-best-parameters-from-grid-search-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Use the best parameters from grid search</a></span></li><li><span><a href="#Randomized-search" data-toc-modified-id="Randomized-search-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Randomized search</a></span></li><li><span><a href="#Feature-Importance" data-toc-modified-id="Feature-Importance-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Feature Importance</a></span></li><li><span><a href="#Feature-Importance-using-eli5" data-toc-modified-id="Feature-Importance-using-eli5-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Feature Importance using eli5</a></span></li></ul></div>

# Imports

In [1]:
import numpy as np
import pandas as pd

import os
import time
import collections
import itertools

# random state
random_state=100
np.random.seed(random_state) # we need this in each cell
np.random.set_state=random_state

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import pickle

# Useful Scripts

In [8]:
def show_method_attributes(obj, ncols=7,start=None, inside=None):
    """ Show all the attributes of a given method.
    Example:
    ========
    show_method_attributes(list)
     """
    lst = [elem for elem in dir(obj) if elem[0]!='_' ]
    lst = [elem for elem in lst 
           if elem not in 'os np pd sys time psycopg2'.split() ]

    if isinstance(start,str):
        lst = [elem for elem in lst if elem.startswith(start)]
        
    if isinstance(start,tuple) or isinstance(start,list):
        lst = [elem for elem in lst for start_elem in start
               if elem.startswith(start_elem)]
        
    if isinstance(inside,str):
        lst = [elem for elem in lst if inside in elem]
        
    if isinstance(inside,tuple) or isinstance(inside,list):
        lst = [elem for elem in lst for inside_elem in inside
               if inside_elem in elem]

    return pd.DataFrame(np.array_split(lst,ncols)).T.fillna('')

def adjustedR2(rsquared,nrows,kcols):
    return rsquared- (kcols-1)/(nrows-kcols) * (1-rsquared)


def multiple_linear_regression(df,features,target,model,
                               verbose=1,cv=5,test_size=0.3):
    """ Multiple Linear Regression Modelling using given model.
    
    Depends:
    Depends on function adjusted r-squared.
    
    
    Returns:
    rmse, r2_train, ar2_train, r2_test, ar2_test, cv
    """
    
    # train test split
    train, test = train_test_split(df, test_size=0.2, random_state=100)

    # train test values
    X = df[features].values
    y = df[target].values.ravel()

    Xtrain = train[features].values
    ytrain = train[target].values.ravel()

    Xtest = test[features].values
    ytest = test[target].values.ravel()
    
    # fitting
    model.fit(Xtrain,ytrain)

    # prediction
    ypreds = model.predict(Xtest)

    # metrics
    rmse = np.sqrt(mean_squared_error(ytest,ypreds)).round(3)
    r2_train = model.score(Xtrain, ytrain).round(3)
    r2_test = model.score(Xtest, ytest).round(3)

    cv = cross_val_score(model, X, y, cv=5,n_jobs=-1,
                         verbose=verbose).mean().round(3)

    ar2_train = adjustedR2(model.score(Xtrain,ytrain),
                           Xtrain.shape[0],
                           len(features)).round(3)
    ar2_test  = adjustedR2(model.score(Xtest,ytest),
                           Xtest.shape[0] ,
                           len(features)).round(3)
    
    return (rmse, r2_train, ar2_train, r2_test, ar2_test, cv)


df_eval = pd.DataFrame({'Model': [],
                           'Details':[],
                           'Root Mean Squared Error (RMSE)':[],
                           'R-squared (training)':[],
                           'Adjusted R-squared (training)':[],
                           'R-squared (test)':[],
                           'Adjusted R-squared (test)':[],
                           '5-Fold Cross Validation':[]})

# Load the data

In [3]:
df = pd.read_csv('../data/raw/kc_house_data.csv')
df.drop(['id','date'],axis=1,inplace=True)
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# Random Forest Modelling

In [4]:
target = ['price']
features = df.drop(['price'],axis=1).columns

In [5]:
model = RandomForestRegressor(n_estimators= 50,random_state=100)

rmse, r2_train, ar2_train, r2_test, ar2_test, cv = \
    multiple_linear_regression(df, features, target,model,
                               verbose=0,test_size=0.2)


df_eval.loc[len(df_eval)] = ['Random Forest Regressor',
                             '', rmse,r2_train,ar2_train,
                             r2_test,ar2_test,cv]


df_eval

,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared (test),5-Fold Cross Validation
0,Random Forest Regressor,,125316.711,0.981,0.981,0.883,0.883,0.872


# Grid Search

Most important hyperparameters of Random Forest:

- n_estimators = n of trees
- max_features = max number of features considered for splitting a node
- max_depth = max number of levels in each decision tree
- min_samples_split = min number of data points placed in a node before the node is split
- min_samples_leaf = min number of data points allowed in a leaf node
- bootstrap = method for sampling data points (with or without replacement)

In [6]:
# train test split
train, test = train_test_split(df, test_size=0.2, random_state=100)

# train test values
X = df[features].values
y = df[target].values.ravel()

Xtrain = train[features].values
ytrain = train[target].values.ravel()

Xtest = test[features].values
ytest = test[target].values.ravel()

In [29]:
%%time
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor(random_state=random_state,n_jobs=-1)


param_grid = [
{'n_estimators': np.arange(20,100,10),
 'max_features': ['sqrt', 'auto', len(features)], 
 'max_depth': [10, 50, None],
 'min_samples_split': [2,5,10],
 'min_samples_leaf': [1,2,4],
 'bootstrap': [True, False]}
]

grid_search_forest = GridSearchCV(model,
                                  param_grid,
                                  cv=5,
                                  n_jobs=-1,
                                  scoring='neg_mean_squared_error',
                                  verbose=2)

grid_search_forest.fit(Xtrain, ytrain)
# this takes very long time, comment this after use
# save the pickled result and load the pickle.

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   43.7s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 41.1min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 59.3min


KeyboardInterrupt: 

In [ ]:
import pickle

with open('../models/grid_search_rf.pkl','wb') as fo:
    pickle.dump(grid_search_forest, fo)

# Use the best parameters from grid search

In [9]:
show_method_attributes(grid_search_forest)

,0,1,2,3,4,5,6
0,best_estimator_,cv,fit,n_jobs,predict,refit_time_,scoring
1,best_index_,cv_results_,get_params,n_splits_,predict_log_proba,return_train_score,set_params
2,best_params_,decision_function,iid,param_grid,predict_proba,score,transform
3,best_score_,error_score,inverse_transform,pre_dispatch,refit,scorer_,verbose
4,classes_,estimator,multimetric_,,,,


In [15]:
df_gs_rf = pd.DataFrame(grid_search_forest.cv_results_)
df_gs_rf.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_bootstrap', 'param_max_depth', 'param_max_features',
       'param_n_estimators', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score'],
      dtype='object')

In [18]:
# df_gs_rf param_max_features

In [23]:
df_gs_rf.pivot_table(
    index=['param_max_features','param_n_estimators'],
    columns=['param_bootstrap','param_max_depth'],
    values='mean_test_score').round(2)\
.style.background_gradient('Blues',axis=None)

In [24]:
grid_search_forest.best_params_

{'bootstrap': True, 'max_depth': 50, 'max_features': 18, 'n_estimators': 40}

In [25]:
grid_search_forest.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
                      max_features=18, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=40,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [26]:
model = RandomForestRegressor(n_estimators= 40,random_state=100, max_features=18,
                             max_depth=50, bootstrap=True)

rmse, r2_train, ar2_train, r2_test, ar2_test, cv = \
    multiple_linear_regression(df, features, target,model,
                               verbose=0,test_size=0.2)


df_eval.loc[len(df_eval)] = ['Random Forest Regressor after grid search',
                             '', rmse,r2_train,ar2_train,
                             r2_test,ar2_test,cv]


df_eval.sort_values('Adjusted R-squared (test)')

,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared (test),5-Fold Cross Validation
0,Random Forest Regressor after grid search,,125118.269,0.981,0.981,0.884,0.883,0.872


# Randomized search

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in  forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]

# max features
max_features = ['auto', 'sqrt']

# max depth of leaves
max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]

# min samples split
min_samples_split = [5, 10]

# random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

pprint(random_grid)

{'max_depth': [1, 23, 45],
 'max_features': ['auto', 'sqrt'],
 'min_samples_split': [5, 10],
 'n_estimators': [20, 65, 110, 155, 200]}


In [12]:
rf_random = RandomizedSearchCV(estimator = model,
                               param_distributions = random_grid,
                               n_iter = 10,
                               cv = 5,
                               verbose=2,
                               random_state=100,
                               n_jobs = -1,
                               scoring='neg_mean_squared_error')
# Fit the random search model
rf_random.fit(Xtrain, ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=50,
                                                   max_features=18,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=40, n_jobs=None,
                                                   oob_score=False,
                                                   random_state=100, verbos

In [13]:
rf_random.best_params_

{'n_estimators': 155,
 'min_samples_split': 10,
 'max_features': 'auto',
 'max_depth': 23}

In [ ]:
model = RandomForestRegressor(n_estimators= 155,
                              random_state=100,
                              max_features='auto',
                              max_depth=23,
                              min_samples_split=10)

rmse, r2_train, ar2_train, r2_test, ar2_test, cv = \
    multiple_linear_regression(df, features, target,model,
                               verbose=0,test_size=0.2)


df_eval.loc[len(df_eval)] = ['Random Forest Regressor after grid search',
                             '', rmse,r2_train,ar2_train,
                             r2_test,ar2_test,cv]


df_eval.sort_values('Adjusted R-squared (test)')

# Feature Importance

In [ ]:
importances = rf_random.best_estimator_.feature_importances_
importances

In [ ]:
df_imp = pd.DataFrame({'feature': features,
                      'importance': importances})

df_imp.sort_values('importance', ascending=False)\
  .style.background_gradient(subset=['importance'])

# Feature Importance using eli5

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
from eli5 import show_prediction

In [ ]:
perm = PermutationImportance(model, random_state=1).fit(Xtest, ytest)

eli5.show_weights(perm, feature_names = features.values.tolist())

In [ ]:
eli5.explain_weights_df(perm, feature_names=features.values.tolist())\
  .style.background_gradient(subset=['weight'])

In [ ]:
test.head()

In [ ]:
eli5.show_prediction(model, test.iloc[0,1:],show_feature_values=True)